This is a classification example to show how to use Oboe for training and testing, in the context of AutoML, i.e., do model selection on the training set and then evaluate the performance of the selected model on the test set.

In [10]:
# necessary modules
import sys
import pandas as pd
import os
import time
import numpy as np
import multiprocessing

#Oboe modules; this will be simplified when Oboe becomes pip installable
automl_path = '../automl/'
sys.path.append(automl_path)
from auto_learner import AutoLearner
import util

#import scikit-learn modules
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# disable warnings
import warnings
warnings.filterwarnings('ignore')

In [11]:
#load and split dataset into training and test folds
data = load_iris()
x = np.array(data['data'])
y = np.array(data['target'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Example 1: a no-brainer use

In [12]:
# initialize the autolearner class
m = AutoLearner(p_type='classification', runtime_limit=10, verbose=False)

In [13]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

In [14]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))    
print("elapsed time: {}".format(elapsed_time))

prediction error: 0.02391304347826088
elapsed time: 9.078747034072876


In [15]:
# get names of the selected machine learning models
m.get_models()

{'ensemble method': 'greedy selection',
 'base learners': {'kSVM': [{'C': 4, 'kernel': 'poly', 'coef0': 10},
   {'C': 8, 'kernel': 'rbf', 'coef0': 10},
   {'C': 2, 'kernel': 'rbf', 'coef0': 0},
   {'C': 0.125, 'kernel': 'rbf', 'coef0': 10}]}}

# Example 2: build an ensemble of models

In [16]:
#experimental settings
VERBOSE = False #whether to print out information indicating current fitting progress
N_CORES = 1 #number of cores
RUNTIME_BUDGET = 15

In [17]:
#optional: limit the types of algorithms
s = ['AB', 'ExtraTrees', 'GNB', 'KNN', 'RF', 'DT']

In [18]:
#autolearner arguments
autolearner_kwargs = {
    'p_type': 'classification',
    'runtime_limit': RUNTIME_BUDGET,
    'verbose': VERBOSE,
    'selection_method': 'min_variance',
    'algorithms': s,
    'stacking_alg': 'greedy',
    'n_cores': N_CORES,
    'build_ensemble': True,
}

In [19]:
#intialize the autolearner class
m = AutoLearner(**autolearner_kwargs)

In [20]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

In [21]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))
print("elapsed time: {}".format(elapsed_time))
print("individual accuracies of selected models: {}".format(m.get_model_accuracy(y_test)))

prediction error: 0.0
elapsed time: 10.359421014785767
individual accuracies of selected models: [0.0, 0.0, 0.02391304347826088]


In [22]:
# get names of the selected machine learning models
m.get_models()

{'ensemble method': 'greedy selection',
 'base learners': {'KNN': [{'n_neighbors': 15, 'p': 1},
   {'n_neighbors': 7, 'p': 1}],
  'ExtraTrees': [{'min_samples_split': 16, 'criterion': 'entropy'}]}}

# Example 3: just select a collection of promising models without building an ensemble afterwards

In [23]:
#experimental settings
VERBOSE = False #whether to print out information indicating current fitting progress
N_CORES = 1 #number of cores
RUNTIME_BUDGET = 15

In [24]:
#optional: limit the types of algorithms
s = ['AB', 'ExtraTrees', 'GNB', 'KNN', 'RF', 'DT']

In [25]:
#autolearner arguments
autolearner_kwargs = {
    'p_type': 'classification',
    'runtime_limit': RUNTIME_BUDGET,
    'verbose': VERBOSE,
    'selection_method': 'min_variance',
    'algorithms': s,
    'stacking_alg': 'greedy',
    'n_cores': N_CORES,
    'build_ensemble': False,
}

In [26]:
#intialize the autolearner class
m = AutoLearner(**autolearner_kwargs)

In [27]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

In [28]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
 
print("elapsed time: {}".format(elapsed_time))
print("accuracies of selected models: {}".format(m.get_model_accuracy(y_test)))

elapsed time: 7.911957025527954
accuracies of selected models: [0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.0, 0.0, 0.0, 0.0, 0.02391304347826088, 0.0, 0.0, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.0, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.0, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.5, 0.5, 0.5, 0.5, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.02391304347826088, 0.04782608695652174, 0.02391304347826088, 0.02391304347826088, 0.0239130434782

Note that we do not have a single accuracy value here if we do not build an ensemble, instead, we just have a collection of fitted models with individual accuracies reported.

In [29]:
# get names of the selected machine learning models
m.get_models()

{'KNN': [{'n_neighbors': 1, 'p': 2},
  {'n_neighbors': 1, 'p': 2},
  {'n_neighbors': 1, 'p': 1},
  {'n_neighbors': 3, 'p': 1},
  {'n_neighbors': 3, 'p': 2},
  {'n_neighbors': 5, 'p': 1},
  {'n_neighbors': 5, 'p': 2},
  {'n_neighbors': 7, 'p': 2},
  {'n_neighbors': 7, 'p': 1},
  {'n_neighbors': 9, 'p': 1},
  {'n_neighbors': 9, 'p': 2},
  {'n_neighbors': 11, 'p': 1},
  {'n_neighbors': 11, 'p': 2},
  {'n_neighbors': 13, 'p': 2},
  {'n_neighbors': 13, 'p': 1},
  {'n_neighbors': 15, 'p': 1},
  {'n_neighbors': 15, 'p': 2}],
 'ExtraTrees': [{'min_samples_split': 2, 'criterion': 'gini'},
  {'min_samples_split': 2, 'criterion': 'entropy'},
  {'min_samples_split': 4, 'criterion': 'gini'},
  {'min_samples_split': 8, 'criterion': 'entropy'},
  {'min_samples_split': 16, 'criterion': 'entropy'},
  {'min_samples_split': 64, 'criterion': 'entropy'},
  {'min_samples_split': 128, 'criterion': 'gini'},
  {'min_samples_split': 4, 'criterion': 'entropy'},
  {'min_samples_split': 8, 'criterion': 'gini'},
  